In [1]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

class Suite2pROICheckGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_CHECK"
        QMainWindow.__init__(self)
        self.widget_manager = WidgetManager()
        self.data_manager = DataManager()
        self.control_manager = ControlManager()
        self.config_manager = ConfigManager()
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.config_manager.setDictTableColumns(key_dict_tablecol=self.app_keys)

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)
        
        # self.initializeAttributes()

        self.setupFileLoadUI()

    """
    setup UI Function
    """
    def setupFileLoadUI(self):
        self.mainLayout = self.makeLayoutMain()
        self.mainLayout.addLayout(self.makeLayoutSectionBottom(), 1, 0, 1, 3)
        
        # bindFunc
        self.bindFuncFileLoadUI()

    # setup
    def setupUI(self):
        try:
            # load data
            self.loadData()
            # make layout
            self.setupLayouts()
            # controls class
            self.setupControls()
            # bindFunc
            self.bindFuncAllWidget()
        except Exception as e:
            return

    def loadData(self):
        self.data_manager.dict_Fall[self.app_keys] = loadFallMAT(
            q_window=self,
            data_manager=self.data_manager,
            key_dict_Fall=self.app_keys,
            key_dict_im_bg=self.app_keys,
            path_fall=self.widget_manager.dict_lineedit[f"{self.app_keys}_path_fall"].text()
        )

    def setupLayouts(self):
        self.mainLayout.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        self.mainLayout.addLayout(self.makeLayoutSectionMiddleUpper(), 0, 1, 1, 1)
        self.mainLayout.addLayout(self.makeLayoutSectionRightUpper(), 0, 2, 1, 1)
        self.widget_manager.dict_table[self.app_keys] = setupWidgetROITable(
                                                                    q_table=self.widget_manager.dict_table[self.app_keys], 
                                                                    len_row=len(self.data_manager.dict_Fall[self.app_keys]["F"]), 
                                                                    dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]
                                                                        )
    def setupControls(self):
        self.widget_manager.dict_table[self.app_keys] = setupWidgetROITable(
            q_table=self.widget_manager.dict_table[self.app_keys], 
            len_row=len(self.data_manager.dict_Fall[self.app_keys]["F"]), 
            dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]
        )
        self.control_manager.table_controls[self.app_keys] = TableControls(
                                                                q_table=self.widget_manager.dict_table[self.app_keys],
                                                                dict_tablecol=self.config_manager.dict_tablecol[self.app_keys],
                                                                key_function_map=self.config_manager.key_function_map
                                                                )
        self.widget_manager.dict_table[self.app_keys].keyPressEvent = lambda event: self.control_manager.table_controls[self.app_keys].keyPressEvent(event)
        self.control_manager.view_controls[self.app_keys] = ViewControls(
                                                                    key_app=self.app_keys,
                                                                    q_view=self.widget_manager.dict_view[self.app_keys], 
                                                                    q_scene=self.widget_manager.dict_scene[self.app_keys], 
                                                                    data_manager=self.data_manager, 
                                                                    widget_manager=self.widget_manager,
                                                                    config_manager=self.config_manager,
                                                                    )

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    # Main Layout
    def makeLayoutMain(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        return QGridLayout(central_widget)

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()

        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)", "Cellpose Mask path (optional)"]
        list_key = [f"{self.app_keys}_path_fall", f"{self.app_keys}_path_reftif", f"{self.app_keys}_path_cellpose"]
        for label, key in zip(list_label, list_key):
            layout.addLayout(makeLayoutLoadFileWidget(self.widget_manager, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    "Left Upper"
    # EventFileの読み込み, plot用
    def makeLayoutComponentEventFilePlot(self):
        layout = QVBoxLayout()
        # Eventのplot range
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                 key_label="eventfile_align_plot_range",
                                                 key_lineedit="eventfile_align_plot_range",
                                                 label="plot range from Event start (pre, post; sec)",
                                                 text_set="(10, 10)"))
        # プロットに重ねるEventFile npyファイルの読み込みボタン, Clearボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(key="loadEventFile", label="Load EventFile npy file"))  ### 要修正
        layout.addWidget(self.widget_manager.makeWidgetButton(key="clearEventFile", label="Clear"))
        return layout

    # trace plotの表示範囲
    def makeLayoutComponentTracePlotRange(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                key_label="minPlotRange",
                                                key_lineedit="minPlotRange",
                                                label="Minimum plot range (sec)",
                                                text_set="30"))
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="plot_eventfile_trace", 
                                                 label="plot EventFile trace", 
                                                 checked=True,
                                                 ))
        return layout

    # 上記二つを合体
    def makeLayoutComponentEventFilePlot_TracePlotRange(self):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentEventFilePlot())
        layout.addLayout(self.makeLayoutComponentTracePlotRange())
        return layout

    "Middle Upper"
    # ROI canvas
    def makeLayoutComponentROIView(self):
        layout = QVBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetView(key=self.app_keys))
        return layout

    # ROI property label, threshold lineedit
    def makeLayoutComponentROIPropertyDisplay_Threshold(self):
        layout = QVBoxLayout()

        layout.addLayout(makeLayoutROIProperty(self.widget_manager, key_label=f"{self.app_keys}_roi_prop"))
        layout.addLayout(makeLayoutROIThresholds(
            self.widget_manager, 
            key_label=f"{self.app_keys}_roi_threshold", 
            key_lineedit=f"{self.app_keys}_roi_threshold", 
            key_checkbox=f"{self.app_keys}_roi_threshold", 
            label_checkbox="ROI Show Threshold", 
            list_threshold_param=["npix", "compact"]))
        return layout

    # ROI display, background image button group, checkbox
    def makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutROITypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.app_keys}_roi_type', dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]))
        layout.addLayout(makeLayoutBGImageTypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.app_keys}_bgimage_type'))
        layout.addLayout(makeLayoutROIChooseSkip(self.widget_manager, key_checkbox=f'{self.app_keys}_roi_skip_choose', dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]))
        return layout

    # channel contrast, ROI opacity slider
    def makeLayoutComponentContrastOpacitySlider(self):
        layout = QVBoxLayout()
        channels = self.config_manager.gui_defaults["CHANNELS"]
        layout_channel = QHBoxLayout()
        for channel in channels:
            layout_channel.addLayout(makeLayoutContrastSlider(self.widget_manager, 
                                                            key_label=f"{self.app_keys}_{channel}", 
                                                            key_checkbox=f"{self.app_keys}_{channel}", 
                                                            key_slider=f"{self.app_keys}_{channel}", 
                                                            label_checkbox=f"Show {channel} channel", 
                                                            label_label=f"{channel} Value", 
                                                            checked=True))

        layout.addLayout(layout_channel)
        layout.addLayout(makeLayoutOpacitySlider(self.widget_manager, 
                                                key_label=self.app_keys, 
                                                key_slider=self.app_keys, 
                                                label=self.app_keys
                                                ))
        return layout

    "Right Upper"
    # Table, ROI count label, Set ROI Celltype, ROICheck IO
    def makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutTableROICountLabel(self.widget_manager, key_label=self.app_keys, key_table=self.app_keys, dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]))
        layout.addLayout(makeLayoutAllROISetSameCelltype(self.widget_manager, key_button=self.app_keys, dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]))
        layout.addLayout(makeLayoutROICheckIO(self.widget_manager, key_button=self.app_keys))
        return layout

    # ROI Filter, threshold
    def makeLayoutComponentROIFilter(self):
        layout = QHBoxLayout()
        layout.addLayout(makeLayoutROIFilterThreshold(self.widget_manager, key_label=self.app_keys, key_lineedit=self.app_keys, dict_roi_threshold=self.config_manager.gui_defaults["ROI_THRESHOLDS"]))
        layout.addLayout(makeLayoutROIFilterButton(self.widget_manager, key_label=self.app_keys, key_button=self.app_keys))
        return layout
    

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 左上
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutCanvasTracePlot(self.widget_manager, 
                                                   key_figure=self.app_keys, 
                                                   key_canvas=self.app_keys, 
                                                   key_app=self.app_keys), 
                        stretch=1)
        layout.addLayout(self.makeLayoutComponentEventFilePlot_TracePlotRange())
        return layout

    # 中上
    def makeLayoutSectionMiddleUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIView())
        layout.addLayout(self.makeLayoutComponentROIPropertyDisplay_Threshold())
        layout.addLayout(self.makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip())
        layout.addLayout(self.makeLayoutComponentContrastOpacitySlider())
        return layout
    
    # 右上
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO())
        layout.addLayout(self.makeLayoutComponentROIFilter())
        return layout

    # 下
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):        
        list_key = [f"{self.app_keys}_path_fall", f"{self.app_keys}_path_reftif", f"{self.app_keys}_path_cellpose"]
        list_filetype = ["mat", "tiff", "npy"]
        for key, filetype in zip(list_key, list_filetype):
            bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.setupUI())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        # ROICheck save load
        bindFuncROICheckIO(
            q_window=self, 
            q_lineedit=self.widget_manager.dict_lineedit[f"{self.app_keys}_path_fall"], 
            q_table=self.widget_manager.dict_table[f"{self.app_keys}"], 
            q_button_save=self.widget_manager.dict_button[f"{self.app_keys}_save_roicheck"], 
            q_button_load=self.widget_manager.dict_button[f"{self.app_keys}_load_roicheck"], 
            dict_tablecol=self.config_manager.dict_tablecol[self.app_keys]
        )
        # ROICheck Table onSelectionChanged
        bindFuncTableSelectionChanged(
            self.widget_manager.dict_table[self.app_keys],
            self.data_manager,
            self.app_keys
        )
        # Radiobutton BGImageType buttonClicked
        bindFuncRadiobuttonBGImageTypeChanged(
            q_buttongroup=self.widget_manager.dict_buttongroup[f"{self.app_keys}_bgimage_type"], 
            data_manager=self.data_manager,
            view_controls=self.control_manager.view_controls[self.app_keys],
            key=self.app_keys
        )

if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

KeyError: 0

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
gui.control_manager.view_controls["pri"].roi_colors

{'pri': {0: (241, 165, 247),
  1: (219, 110, 252),
  2: (250, 137, 153),
  3: (165, 255, 243),
  4: (127, 162, 217),
  5: (215, 165, 177),
  6: (184, 207, 147),
  7: (203, 196, 198),
  8: (187, 202, 188),
  9: (171, 115, 122),
  10: (240, 226, 116),
  11: (110, 182, 108),
  12: (119, 157, 206),
  13: (202, 124, 171),
  14: (106, 226, 104),
  15: (105, 141, 172),
  16: (239, 214, 157),
  17: (137, 246, 227),
  18: (150, 204, 129),
  19: (233, 243, 156),
  20: (102, 114, 104),
  21: (135, 109, 152),
  22: (126, 159, 218),
  23: (245, 136, 148),
  24: (198, 182, 117),
  25: (201, 199, 253),
  26: (127, 214, 230),
  27: (123, 137, 114),
  28: (124, 139, 235),
  29: (242, 160, 223),
  30: (190, 251, 133),
  31: (136, 203, 177),
  32: (252, 252, 102),
  33: (110, 141, 114),
  34: (239, 124, 158),
  35: (184, 142, 186),
  36: (228, 216, 183),
  37: (128, 184, 216),
  38: (163, 238, 242),
  39: (238, 221, 174),
  40: (107, 190, 109),
  41: (134, 223, 221),
  42: (248, 170, 227),
  43: (170, 11

In [5]:
gui.control_manager.view_controls["pri"].getROIColor()

KeyError: '0'